### Pandas Overview
This tutorial is adapted from the followin [reference](https://medium.com/@ulriktpedersen/15-pandas-tips-step-up-your-dataframe-game-bb413e6c2138) please checkout that site for more explanation and tips on working with Pandas dataframes. 

Also you can always refer to the Pandas [Documentation](https://pandas.pydata.org/docs/) for complete documentation of the library and more examples. 

Here we will walk through some of the most commonly used pandas functions to get you familiar with the libraries capabilities. 

To get started we'll use one of our streamflow CSV files as an example.

In [2]:
import pandas as pd
import numpy as np
import datetime

# Read in the streamflow csv as a pandas dataframe
data = pd.read_table('../../data/streamflow_week5.txt', sep='\t', skiprows=31,
                     names=['agency_cd', 'site_no', 'datetime', 'flow', 'code'],parse_dates=['datetime'] )
data.set_index('datetime', inplace=True)

# extract the year, month, and day
data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day

### Use the Head and Tail functions to quickly look at the data
The head() and tail() functions allow you to quickly view the first and last few rows of a data frame, respectively. This is useful for getting a quick overview of the data and checking if it's been loaded correctly.

**NOTE the syntax here**: Head and tail are both methods are they are linked with the object. So we call them by `data_object_name.method()` in this case our object is called `data` so we say `data.head()` if we had named our dataset above `stream_data` instead then this call would be `stream_data.head()`

In [3]:
# view the first 5 rows
print("Here are the first five rows of my dataframe")
print(data.head())

# view the last 5 rows
print("Here are the last five rows of my dataframe")
print(data.tail())


Here are the first five rows of my dataframe
           agency_cd  site_no   flow code  year  month  day
datetime                                                   
1989-01-01      USGS  9506000  207.0    A  1989      1    1
1989-01-02      USGS  9506000  205.0    A  1989      1    2
1989-01-03      USGS  9506000  205.0    A  1989      1    3
1989-01-04      USGS  9506000  232.0    A  1989      1    4
1989-01-05      USGS  9506000  259.0    A  1989      1    5
Here are the last five rows of my dataframe
           agency_cd  site_no  flow code  year  month  day
datetime                                                  
2023-09-19      USGS  9506000  81.9    P  2023      9   19
2023-09-20      USGS  9506000  74.5    P  2023      9   20
2023-09-21      USGS  9506000  73.2    P  2023      9   21
2023-09-22      USGS  9506000  66.8    P  2023      9   22
2023-09-23      USGS  9506000  71.1    P  2023      9   23


## Summarize the dataframe
The methods `describe` and `info` provide easy and helpful summaries of the data

In [4]:
print("This is the info")
print(data.info())

print("This is what describe gives you")
print(data.describe())

This is the info
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12684 entries, 1989-01-01 to 2023-09-23
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   agency_cd  12684 non-null  object 
 1   site_no    12684 non-null  int64  
 2   flow       12667 non-null  float64
 3   code       12667 non-null  object 
 4   year       12684 non-null  int32  
 5   month      12684 non-null  int32  
 6   day        12684 non-null  int32  
dtypes: float64(1), int32(3), int64(1), object(2)
memory usage: 644.1+ KB
None
This is what describe gives you
         site_no          flow          year         month           day
count    12684.0  12667.000000  12684.000000  12684.000000  12684.000000
mean   9506000.0    353.192587   2005.867313      6.489357     15.721854
std          0.0   1464.717308     10.025937      3.439771      8.798025
min    9506000.0     19.000000   1989.000000      1.000000      1.000000
25%    9506000.0     9

### Selecting Data with Loc and Iloc
- `loc[]` and `iloc[]` are two methods for selecting data from a data frame. `loc[]` is used for label-based indexing, while `iloc[]` is used for integer-based indexing. Understanding these methods is essential for selecting specific rows and columns from a data frame.
- Syntax note: here too these are methods associated with the object so the object name comes first. Note though that we use `[]` because these are indexing the dataframe instead of the usual `()` that we use for functions and methods. 

In [5]:
# select a full by row by its index value using loc[]
print("Full row select using loc")
print(data.loc["1989-01-03"])

#select a all the rows where the column values meet some criteria
print('Single Column select using loc')
print(data.loc[data['day'] <= 10])

#select rows and columns using iloc[]
#print("Selct using iloc")
#print(data.iloc[0:5, 0:3])  


Full row select using loc
agency_cd       USGS
site_no      9506000
flow           205.0
code               A
year            1989
month              1
day                3
Name: 1989-01-03 00:00:00, dtype: object
Single Column select using loc
           agency_cd  site_no   flow code  year  month  day
datetime                                                   
1989-01-01      USGS  9506000  207.0    A  1989      1    1
1989-01-02      USGS  9506000  205.0    A  1989      1    2
1989-01-03      USGS  9506000  205.0    A  1989      1    3
1989-01-04      USGS  9506000  232.0    A  1989      1    4
1989-01-05      USGS  9506000  259.0    A  1989      1    5
...              ...      ...    ...  ...   ...    ...  ...
2023-09-06      USGS  9506000  105.0    P  2023      9    6
2023-09-07      USGS  9506000   88.4    P  2023      9    7
2023-09-08      USGS  9506000    NaN  NaN  2023      9    8
2023-09-09      USGS  9506000    NaN  NaN  2023      9    9
2023-09-10      USGS  9506000    Na

### Grouping and Aggregating Data
Grouping and aggregating data are powerful functions in Pandas. `groupby()` allows you to group data by one or more columns, while `agg()` calculates summary statistics for each group. These functions are useful for summarizing and analyzing large data sets.

The syntax for these two approaches is as follows: 
group data by a column and calculate the mean:

```grouped_df = df.groupby('column_name').agg({'column_to_agg': 'mean'})```

group data by multiple columns and calculate the sum:

```grouped_df = df.groupby(['column1', 'column2']).agg({'column_to_agg': 'sum'})```


In [6]:
monthly_sum = data.groupby(['year', 'month']).agg({'flow': 'mean'})
print(monthly_sum)




                  flow
year month            
1989 1      214.064516
     2      263.035714
     3      299.387097
     4      126.433333
     5       87.774194
...                ...
2023 5      128.903226
     6       70.896667
     7       46.331818
     8       84.300000
     9       92.886667

[417 rows x 1 columns]


In [7]:
#Example using group and describe
#monthly_data = data.groupby('month')[["flow"]].describe()
#print(monthly_data)

#A bunch of different examples using groupby and agg together. Add print statments thorughout to see what each of these do. 
monthly_sum = data.groupby('month').agg('sum')

monthly_sum = data.groupby(['month']).agg({'flow': 'min'})

monthly_sum = data.groupby(['month', 'day']).agg(
    {'flow': 'min'})

monthly_sum = data.groupby(['month', 'day']).agg('mean', numeric_only=True)
monthly_sum = data.groupby(['month', 'day']).agg({'flow': ['min', 'max'], 'year': 'min'})

print(monthly_sum)



            flow           year
             min      max   min
month day                      
1     1    175.0   4980.0  1989
      2    173.0   5410.0  1989
      3    172.0   2660.0  1989
      4    172.0   8130.0  1989
      5    170.0   5700.0  1989
...          ...      ...   ...
12    27   169.0    899.0  1989
      28   167.0   1320.0  1989
      29   177.0   7430.0  1989
      30   176.0  28700.0  1989
      31   173.0   6900.0  1989

[366 rows x 3 columns]


In [11]:
monthly_sum = data.groupby(['month', 'day']).agg('mean', numeric_only=True)
print(monthly_sum)

             site_no         flow    year
month day                                
1     1    9506000.0   540.428571  2006.0
      2    9506000.0   590.000000  2006.0
      3    9506000.0   403.828571  2006.0
      4    9506000.0   528.228571  2006.0
      5    9506000.0   462.942857  2006.0
...              ...          ...     ...
12    27   9506000.0   286.852941  2005.5
      28   9506000.0   302.088235  2005.5
      29   9506000.0   660.382353  2005.5
      30   9506000.0  1277.941176  2005.5
      31   9506000.0   578.441176  2005.5

[366 rows x 3 columns]


In [1]:
monthly_sum = data.groupby(['month']).agg('mean', numeric_only=True)

NameError: name 'data' is not defined